In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
df = pd.read_csv('/content/fake_real.csv', encoding='utf-8')
df.head()

,Unnamed: 0,Content,Title,Type
0,0,new york reuters us environmental group sierra...,politicsNews,true
1,1,washington reuters us air force asked industry...,politicsNews,true
2,2,saturday paul ryan posted photo instagram phot...,News,fake
3,3,america keeps waiting word hillary indicted ob...,politics,fake
4,4,religion peace ht weasel zippers,left-news,fake


In [5]:
# Define a mapping dictionary
type_mapping = {'fake': 0, 'true': 1}

# Map the 'Type' column using the mapping dictionary
df['Type'] = df['Type'].map(type_mapping)

In [6]:
df.head()

,Unnamed: 0,Content,Title,Type
0,0,new york reuters us environmental group sierra...,politicsNews,1.0
1,1,washington reuters us air force asked industry...,politicsNews,1.0
2,2,saturday paul ryan posted photo instagram phot...,News,0.0
3,3,america keeps waiting word hillary indicted ob...,politics,0.0
4,4,religion peace ht weasel zippers,left-news,0.0


In [7]:
df.shape

(18335, 4)

In [8]:
# Remove HTML tags
df['Content'] = df['Content'].replace(r'<[^>]+>', '', regex=True)
# Lowercase the text
df['Content'] = df['Content'].str.lower()

In [10]:
# Step 4: Feature Engineering
from sklearn.feature_extraction.text import TfidfVectorizer

# Drop rows with missing values in the 'Content' column
df = df.dropna(subset=['Content'])

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the 'Content' column
X = tfidf_vectorizer.fit_transform(df['Content'])
y = df['Type']

In [11]:
# Step 5: Splitting the Data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Step 6: Model Training
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [13]:
# Step 7: Model Evaluation
accuracy = clf.score(X_test, y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9803758982863461


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a pipeline with TF-IDF vectorizer and logistic regression classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression())
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'tfidf__max_features': [1000, 2000, 3000],  # Number of features in TF-IDF vectorizer
    'tfidf__ngram_range': [(1, 1), (1, 2)],      # n-gram range for TF-IDF vectorizer
    'clf__C': [0.1, 1, 10]                       # Regularization parameter for logistic regression
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)

# Split the dataset into features and target variable
X = df['Content']
y = df['Type']

# Perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best CV accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
